In [3]:
import requests
from bs4 import BeautifulSoup
import json
from datetime import date
from pymongo import MongoClient

In [32]:
url = "http://www.pixiv.net/ranking.php?mode=daily"
response = requests.get(url)
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36', 
          'password': 'rd.)T6vF',
          'pixiv_id' : 'flicfloclol@hotmail.fr'}
response = requests.get(url, headers=headers, timeout = 10)

parse = url.split('&')

imageId = []


if (len(parse) == 1):
    rankingDate = str(date.today())   
else:
    rankingDate = parse[1][5:9] + '-' + parse[1][9:11] + '-' + parse[1][11:13]


soup = BeautifulSoup(response.text)
links = []

# for single illustration (different class)
for link in soup.find_all('a', class_="work _work"):
    links.append("https://www.pixiv.net/ajax/illust/" + link.get('href').split('/')[2] + "?lang=en")
    imageId.append(link.get('href').split('/')[2])
    
# for multiple illustrations (different class)\n",
for link in soup.find_all('a', class_="work _work multiple"):
    links.append("https://www.pixiv.net/ajax/illust/" + link.get('href').split('/')[2] + "?lang=en")
    imageId.append(link.get('href').split('/')[2])



In [33]:
#url = "https://www.pixiv.net/ajax/ill?lang=enust/89101579"

data = []

counter = 0
imageCounter = 0

for url in links:

    response = requests.get(url, headers=headers, timeout = 100)
    jsonData = json.loads(response.text)

    tags = []

    for i in jsonData['body']['tags']['tags']:
        data.append({})
        if 'translation' not in i:
            if 'romaji' not in i:
                data[counter].update({'tag': i['tag']})
            else:
                data[counter].update({'tag': i['romaji']})
        else:
            data[counter].update({'tag': i['translation']['en']})
            
        data[counter].update({'date' : rankingDate})
        data[counter].update({'username' : jsonData['body']['userName']})
        data[counter].update({'imageId' : imageId[imageCounter]})
        counter += 1
    imageCounter += 1
    

print(data)


[{'tag': 'VOCALOID', 'date': '2021-04-17', 'username': 'おむたつ/omutatsu', 'imageId': '89155451'}, {'tag': 'hatsune miku', 'date': '2021-04-17', 'username': 'おむたつ/omutatsu', 'imageId': '89155451'}, {'tag': 'ODDS&ENDS', 'date': '2021-04-17', 'username': 'おむたつ/omutatsu', 'imageId': '89155451'}, {'tag': 'korehaiihatsune', 'date': '2021-04-17', 'username': 'おむたつ/omutatsu', 'imageId': '89155451'}, {'tag': 'Miku must be an angel', 'date': '2021-04-17', 'username': 'おむたつ/omutatsu', 'imageId': '89155451'}, {'tag': 'Vocaloid 10000+ bookmarks', 'date': '2021-04-17', 'username': 'おむたつ/omutatsu', 'imageId': '89155451'}, {'tag': 'Gold Ship', 'date': '2021-04-17', 'username': 'Nardack', 'imageId': '89174625'}, {'tag': 'gorushi', 'date': '2021-04-17', 'username': 'Nardack', 'imageId': '89174625'}, {'tag': 'Uma Musume Pretty Derby', 'date': '2021-04-17', 'username': 'Nardack', 'imageId': '89174625'}, {'tag': 'horse girl', 'date': '2021-04-17', 'username': 'Nardack', 'imageId': '89174625'}, {'tag': 'high 

In [34]:
client = MongoClient('mongo')

cursor = collection.find({"date": rankingDate}, {"imageId", 'tag'})
ids = []
for document in cursor:
    ids.append(document)

database = client['local']
collection = database['pixitag']
for i in data:
    if i:
        for dbData in ids:
            if dbData['imageId'] == i['imageId'] and dbData['tag'] == i['tag']:
                i = {}
                break
        if i:
            collection.insert_one(i)
            print('Insert: ' + i['imageId'])


Insert: 89155451
Insert: 89155451
Insert: 89155451
Insert: 89155451
Insert: 89155451
Insert: 89155451
Insert: 89174625
Insert: 89174625
Insert: 89174625
Insert: 89174625
Insert: 89174625
Insert: 89174625
Insert: 89174625
Insert: 89174625
Insert: 89174625
Insert: 89174736
Insert: 89174736
Insert: 89174736
Insert: 89174736
Insert: 89174736
Insert: 89174736
Insert: 89174736
Insert: 89174736
Insert: 89174736
Insert: 89174736
Insert: 89155465
Insert: 89155465
Insert: 89155465
Insert: 89155465
Insert: 89160645
Insert: 89160645
Insert: 89155499
Insert: 89155499
Insert: 89174443
Insert: 89174443
Insert: 89174443
Insert: 89174443
Insert: 89174443
Insert: 89174443
Insert: 89174443
Insert: 89162822
Insert: 89162822
Insert: 89162822
Insert: 89162822
Insert: 89162822
Insert: 89162822
Insert: 89155482
Insert: 89155482
Insert: 89155482
Insert: 89155482
Insert: 89155494
Insert: 89155494
Insert: 89155494
Insert: 89155494
Insert: 89174996
Insert: 89179417
Insert: 89179417
Insert: 89175768
Insert: 891757

In [ ]:
client = MongoClient('mongo')
database = client['local']
collection = database['pixitag']
cursor = collection.find({'tag': 'original'}, {'imageId', 'tag'})
ids = []
for document in cursor:
    ids.append(document)
    print(document)
    print('--------')

In [18]:
collection.remove()
print(collection)

Collection(Database(MongoClient(host=['mongo:27017'], document_class=dict, tz_aware=False, connect=True), 'local'), 'pixitag')


<ipython-input-18-ea60bee885b1>:1: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  collection.remove()


In [85]:
#Get most popular tag by date
cursor = collection.aggregate([
    {"$match": {'date': '2021-04-17'}},
    {'$group': {
        '_id': '$tag',
        'nb': {'$sum': 1}
    }},
    {'$sort': {'nb': -1}}
])
for document in cursor:
    ids.append(document)
    print(document)

{'_id': 'original', 'nb': 14}
{'_id': 'girl', 'nb': 6}
{'_id': 'Uma Musume Pretty Derby', 'nb': 5}
{'_id': 'Fate/GrandOrder', 'nb': 4}
{'_id': 'FGO', 'nb': 4}
{'_id': 'original 5000+ bookmarks', 'nb': 3}
{'_id': 'Virtual Youtuber', 'nb': 3}
{'_id': 'clothed breasts', 'nb': 3}
{'_id': 'Gold Ship (UMPD)', 'nb': 3}
{'_id': 'evangelion', 'nb': 3}
{'_id': 'Fate/GO 1000+ bookmarks', 'nb': 3}
{'_id': 'umaamusumepurithi-da-bi-sennyu-za-iri', 'nb': 2}
{'_id': 'Genshin Impact', 'nb': 2}
{'_id': 'subarashikihokkorinosekai', 'nb': 2}
{'_id': 'Evangelion New Theatrical Release', 'nb': 2}
{'_id': 'Evangelion 1000+ bookmarks', 'nb': 2}
{'_id': 'evagosennyu-za-zuiri', 'nb': 2}
{'_id': 'Shikinami Asuka Langley', 'nb': 2}
{'_id': 'Pokémon', 'nb': 2}
{'_id': 'Ashiya Douman (Fate)', 'nb': 2}
{'_id': 'Nijisanji', 'nb': 2}
{'_id': 'creation', 'nb': 2}
{'_id': 'korekosogago-rudoshippu', 'nb': 2}
{'_id': 'Project Sekai', 'nb': 2}
{'_id': 'work illustration', 'nb': 2}
{'_id': 'plugsuit', 'nb': 2}
{'_id': 'Lumi

In [115]:
#Get most 
cursor = collection.aggregate([
    #{"$match": {'date': '2021-04-17'}},
    {'$group': {
        '_id': {'username': '$username'},
        'images': {'$addToSet': '$imageId'}
    }},
    {'$project': {'nb': {'$size': '$images'}}},
    {'$sort': {'nb': -1}}
])
for document in cursor:
    ids.append(document)
    print(document)

{'_id': {'username': 'チャイ'}, 'nb': 5}
{'_id': {'username': 'ポ～ン（出水ぽすか）'}, 'nb': 5}
{'_id': {'username': '寺田てら'}, 'nb': 3}
{'_id': {'username': 'TOKKYU'}, 'nb': 3}
{'_id': {'username': 'スコッティ'}, 'nb': 3}
{'_id': {'username': 'かわやばぐ'}, 'nb': 2}
{'_id': {'username': 'sakimichan'}, 'nb': 2}
{'_id': {'username': 'ぽむ'}, 'nb': 2}
{'_id': {'username': '上山道郎'}, 'nb': 2}
{'_id': {'username': 'AU'}, 'nb': 2}
{'_id': {'username': '村カルキ'}, 'nb': 2}
{'_id': {'username': 'Saclia'}, 'nb': 2}
{'_id': {'username': 'EB十'}, 'nb': 2}
{'_id': {'username': 'きの'}, 'nb': 1}
{'_id': {'username': '旧都なぎ'}, 'nb': 1}
{'_id': {'username': 'Aoi Ogata'}, 'nb': 1}
{'_id': {'username': 'あるてら'}, 'nb': 1}
{'_id': {'username': '河合朗🎀短編集発売中'}, 'nb': 1}
{'_id': {'username': 'Rosuuri'}, 'nb': 1}
{'_id': {'username': 'ハラダミユキ'}, 'nb': 1}
{'_id': {'username': '色素'}, 'nb': 1}
{'_id': {'username': 'Nengoro(ネんごろぅ)'}, 'nb': 1}
{'_id': {'username': 'かれい'}, 'nb': 1}
{'_id': {'username': '花ヶ田 / hanagata'}, 'nb': 1}
{'_id': {'username': 

In [111]:
cursor = collection.distinct('imageId', {'username': 'チャイ'})
ids = []
for document in cursor:
    ids.append(document)
    print(document)

88835022
88860800
89150224
89169421
89188590
